In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10):
    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []

    for epoch in range(epochs):
        model.train()
        running_loss, correct = 0.0, 0

        print(f"Epoch {epoch + 1}/{epochs}")
        for inputs, labels in tqdm(train_loader, desc="Training"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct / len(train_loader.dataset)
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

        val_loss, val_accuracy = validate_model(model, criterion, val_loader)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    metrics = {
        "train_losses": train_losses,
        "train_accuracies": train_accuracies,
        "val_losses": val_losses,
        "val_accuracies": val_accuracies
    }
    return model, metrics

def validate_model(model, criterion, loader):
    model.eval()
    running_loss, correct = 0.0, 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()

    val_loss = running_loss / len(loader.dataset)
    val_accuracy = correct / len(loader.dataset)
    return val_loss, val_accuracy

def evaluate_model(model, loader, output_matrix_path):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    classes = loader.dataset.dataset.classes if isinstance(loader.dataset, torch.utils.data.Subset) else loader.dataset.classes

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.savefig(output_matrix_path)
    plt.close()

    return all_preds, all_labels, cm

def plot_metrics(metrics, output_dir):
    epochs = range(1, len(metrics["train_losses"]) + 1)

    # Plot Loss
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, metrics["train_losses"], label="Training Loss")
    plt.plot(epochs, metrics["val_losses"], label="Validation Loss")
    plt.title("Loss per Epoch")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_dir, "loss_plot.png"))
    plt.close()

    # Plot Accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, metrics["train_accuracies"], label="Training Accuracy")
    plt.plot(epochs, metrics["val_accuracies"], label="Validation Accuracy")
    plt.title("Accuracy per Epoch")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_dir, "accuracy_plot.png"))
    plt.close()

if __name__ == '__main__':
    output_dir = "Arqumlatestoutput"
    os.makedirs(output_dir, exist_ok=True)

    model_save_path = os.path.join(output_dir, "cnn_model.pth")
    train_conf_matrix_path = os.path.join(output_dir, "train_confusion_matrix.png")
    val_conf_matrix_path = os.path.join(output_dir, "val_confusion_matrix.png")
    test_conf_matrix_path = os.path.join(output_dir, "test_confusion_matrix.png")
    classification_report_path = os.path.join(output_dir, "classification_report.txt")

    train_transform = transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(224),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.GaussianBlur(3),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

    test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

    train_dataset = datasets.ImageFolder(root=r'C:\Users\Ibad\Downloads\AI Open\AI Open\CT-Scan\CT-Scan Train', transform=train_transform)
    test_dataset = datasets.ImageFolder(root=r'C:\Users\Ibad\Downloads\AI Open\AI Open\CT-Scan\CT-Scan Test', transform=test_transform)

    validation_split = 0.2
    train_size = int((1 - validation_split) * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

    model = models.resnet50(weights="IMAGENET1K_V1")
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, len(train_dataset.dataset.classes))
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model, metrics = train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10)
    plot_metrics(metrics, output_dir)

    train_preds, train_labels, train_cm = evaluate_model(model, train_loader, train_conf_matrix_path)
    val_preds, val_labels, val_cm = evaluate_model(model, val_loader, val_conf_matrix_path)
    test_preds, test_labels, test_cm = evaluate_model(model, test_loader, test_conf_matrix_path)

    with open(classification_report_path, "w") as f:
        train_report = classification_report(train_labels, train_preds, target_names=train_dataset.dataset.classes)
        val_report = classification_report(val_labels, val_preds, target_names=train_dataset.dataset.classes)
        test_report = classification_report(test_labels, test_preds, target_names=test_dataset.classes)

        f.write("Classification Report - Training:\n")
        f.write(train_report + "\n")
        f.write("\nClassification Report - Validation:\n")
        f.write(val_report + "\n")
        f.write("\nClassification Report - Testing:\n")
        f.write(test_report)

    torch.save(model.state_dict(), model_save_path)
    print(f"Model and plots saved in {output_dir}")


Epoch 1/10


Training: 100%|████████████████████████████████████████████████████████████████████████| 91/91 [00:53<00:00,  1.69it/s]


Train Loss: 0.7445, Train Accuracy: 0.7044
Validation Loss: 5.5164, Validation Accuracy: 0.2784
Epoch 2/10


Training: 100%|████████████████████████████████████████████████████████████████████████| 91/91 [00:47<00:00,  1.90it/s]


Train Loss: 0.4817, Train Accuracy: 0.8157
Validation Loss: 0.4189, Validation Accuracy: 0.8421
Epoch 3/10


...

Train Loss: 0.2169, Train Accuracy: 0.9269
Validation Loss: 0.4019, Validation Accuracy: 0.8740
Epoch 10/10


Training: 100%|████████████████████████████████████████████████████████████████████████| 91/91 [00:49<00:00,  1.85it/s]


Train Loss: 0.2072, Train Accuracy: 0.9290
Validation Loss: 0.7152, Validation Accuracy: 0.8352


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 32/32 [00:26<00:00,  1.22it/s]


Model and plots saved in Arqumlatestoutput
